In [1]:
# Some initial setting:
storageAccountName = "mihandd2020export"
storageAccountKey = "<secret>"
storageContainer = "8433a4f3-af6b-4195-ad5d-0635fb96490e"

In [2]:
dbutils.fs.mount(
  source = "wasbs://" + storageContainer + "@" + storageAccountName + ".blob.core.windows.net",
  mount_point = '/mnt/dataexport',
  extra_configs = {"fs.azure.account.key." + storageAccountName + ".blob.core.windows.net":storageAccountKey})

Out[2]: True

In [3]:
%sql CREATE TEMPORARY TABLE observationTable USING json OPTIONS (path "/mnt/dataexport/Observation.ndjson")

In [4]:
%sql CREATE TEMPORARY TABLE patientTable USING json OPTIONS (path "/mnt/dataexport/Patient.ndjson")

In [5]:
%sql 
CREATE OR REPLACE TEMPORARY VIEW temp_heights AS 
  SELECT * FROM (
    SELECT 
      SUBSTRING_INDEX(subject.reference,'/',-1) AS patient, 
      valueQuantity.value as heightValue, 
      valueQuantity.unit as heightUnit,  
      ROW_NUMBER() OVER (PARTITION BY subject.reference ORDER BY issued DESC) AS rn 
   FROM observationTable WHERE code.coding[0].code = "8302-2") tm 
  WHERE tm.rn = 1

In [6]:
%sql 
CREATE OR REPLACE TEMPORARY VIEW temp_weights AS 
  SELECT * FROM (
    SELECT 
      SUBSTRING_INDEX(subject.reference,'/',-1) AS patient, 
      valueQuantity.value as weightValue, 
      valueQuantity.unit as weightUnit,  
      ROW_NUMBER() OVER (PARTITION BY subject.reference ORDER BY issued DESC) AS rn 
   FROM observationTable WHERE code.coding[0].code = "29463-7") tm 
  WHERE tm.rn = 1

In [7]:
%sql 
CREATE OR REPLACE TEMPORARY VIEW temp_latitude AS 
  SELECT id, coord.valueDecimal AS latitude FROM 
    (SELECT id, explode(address[0].extension[0].extension) as coord FROM patientTable) 
  WHERE coord.url = 'latitude';
  
CREATE OR REPLACE TEMPORARY VIEW temp_longitude AS 
  SELECT id, coord.valueDecimal AS longitude FROM 
    (SELECT id, explode(address[0].extension[0].extension) as coord FROM patientTable) 
  WHERE coord.url = 'longitude'

In [8]:
%sql 
SELECT
  patientTable.id, 
  patientTable.name[0].family AS lastName, 
  temp_longitude.longitude AS longitude, 
  temp_latitude.latitude AS latitude, 
  temp_weights.weightValue, temp_heights.heightValue
FROM patientTable 
  INNER JOIN temp_weights ON temp_weights.patient = patientTable.id 
  INNER JOIN temp_heights ON temp_heights.patient = patientTable.id 
  INNER JOIN temp_latitude ON temp_latitude.id = patientTable.id 
  INNER JOIN temp_longitude ON temp_longitude.id = patientTable.id
LIMIT 10

id,lastName,longitude,latitude,weightValue,heightValue
594d4283-3acb-4f46-a97e-3e845b5b7b1e,Cassin499,-70.99855518177093,41.9768387721285,90.7,178.3
e7a85ddd-af43-4955-9167-1f0418b37997,Walter473,-70.90101002669104,42.48264956980514,108.2,188.5
a32c525e-5af4-4dc2-a6d9-846b68da7cec,Murray856,-71.20926478303276,42.3904493291209,91.1,173.3
2d4a9f5a-d85f-4cf3-be3f-c7acc6ff7b81,Rath779,-70.06193778174459,41.772652063512545,18.7,107.7
e3c52f43-744e-4500-897c-7b7fdf82fb87,Vega377,-70.9834912732618,42.468078587899306,86.7,169.7
c13d3170-7abd-4bdd-83eb-611adb9c54fd,Koepp521,-70.64738046500526,41.73272841490259,85.2,174.8
b09d027b-24a6-41e7-9773-d2ea0eb37be2,Upton904,-71.19959372945458,42.6974579230271,88.2,170.8
79704b8c-86b3-404b-972d-2d35d84edb8c,Carter549,-71.17372282029753,42.66658230787945,24.2,105.9
1f17a519-a9af-4a3a-9088-0c8ac7359537,Predovic534,-70.8708742118531,42.085334883538984,88.6,175.2
32bdfa6a-acc7-4332-9993-493d35633e46,Medhurst46,-71.27050665939483,42.10683810820223,104.3,190.2
